In [1]:
#import sys
#sys.path.append("d:\\OneDrive - Universiteit Utrecht\\Documents\\000 - Documenti\\PROJECTS\\x-tree-search")

In [2]:
from games.tic_tac_toe import TicTacToe, simple_scoring_function
from src.game.game import GameAgent, User
from algorithms.minimax import MiniMax
from explainers.minimax_explainer import MiniMaxExplainer

# Define game

In [3]:
game = TicTacToe()

# Define tic-tac-toe scoring function for the minimax

In [4]:
explainer = MiniMaxExplainer()

In [5]:
explainer.available_frameworks()

lowlevel
highlevel


# Play and Explain

In [6]:
opponent = GameAgent(agent_id=0, core=MiniMax(simple_scoring_function, max_depth=6, use_alpha_beta=False))
user = User(agent_id=1, ask_what = False)

In [7]:
game.set_players([opponent, user])

game.act({'where' : (0,0), 'who': 1}) # First move by user

In [8]:
await game.start_on_jupyter()

In [9]:
explainer.explain_adjective(opponent.choice, 'the best', explanation_depth=3, with_framework='highlevel')

Given:
 [['O' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

[['O' ' ' ' ']
 [' ' 'X' ' ']
 [' ' ' ' ' ']] X in (1, 1), id=0_3 is the best (because
 
	[['O' ' ' ' ']
	 [' ' 'X' ' ']
	 [' ' ' ' ' ']] X in (1, 1), id=0_3 has as possible alternative moves 
	[['O' 'X' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] X in (0, 1), id=0_0, 
	[['O' ' ' 'X']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] X in (0, 2), id=0_1, 
	[['O' ' ' ' ']
	 ['X' ' ' ' ']
	 [' ' ' ' ' ']] X in (1, 0), id=0_2 (only showing relevant 3)
	and 
	[['O' ' ' ' ']
	 [' ' 'X' ' ']
	 [' ' ' ' ' ']] X in (1, 1), id=0_3 is better than them (because
	 	it leads to a better position (because
 
			[['O' ' ' ' ']
			 [' ' 'X' ' ']
			 [' ' ' ' ' ']] X in (1, 1), id=0_3 has as next possible move 
			[['O' 'O' ' ']
			 [' ' 'X' ' ']
			 [' ' ' ' ' ']] O in (0, 1), id=0_3_0
			and 
			[['O' 'X' ' ']
			 [' ' ' ' ' ']
			 [' ' ' ' ' ']] X in (0, 1), id=0_0 has as next possible move 
			[['O' 'X' ' ']
			 ['O' ' ' ' ']
			 [' ' ' ' ' ']] O in (1, 0), id=0_0_1
	

In [10]:
explainer.explain_adjective(opponent.choice, 'best', explanation_depth=4, with_framework='lowlevel')

Given:
 [['O' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

[['O' ' ' ' ']
 [' ' 'X' ' ']
 [' ' ' ' ' ']] X in (1, 1), id=0_3 is best ←
 	(assumption) By definition a node is "best" if it's "better than" than all "siblings"
	∧ 
	[['O' ' ' ' ']
	 [' ' 'X' ' ']
	 [' ' ' ' ' ']] X in (1, 1), id=0_3 has siblings = 
	[['O' 'X' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] X in (0, 1), id=0_0, 
	[['O' ' ' 'X']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] X in (0, 2), id=0_1, 
	[['O' ' ' ' ']
	 ['X' ' ' ' ']
	 [' ' ' ' ' ']] X in (1, 0), id=0_2, 
	[['O' ' ' ' ']
	 [' ' ' ' 'X']
	 [' ' ' ' ' ']] X in (1, 2), id=0_4, 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 ['X' ' ' ' ']] X in (2, 0), id=0_5, 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' 'X' ' ']] X in (2, 1), id=0_6, 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'X']] X in (2, 2), id=0_7 ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ 
	[['O' ' ' ' ']
	 [' ' 'X' ' ']
	 [' ' ' ' ' ']] X in (1, 1), id=0_3 is better than them ←
	 		(assumption) By

In [11]:
explainer.del_explanation_tactic('OnlyRelevantComparisons', to_framework='highlevel', to_adjective='the best')

In [12]:
explainer.explain_adjective(opponent.choice, 'best', explanation_depth=2)

Given:
 [['O' ' ' ' ']
 [' ' ' ' ' ']
 [' ' ' ' ' ']]

[['O' ' ' ' ']
 [' ' 'X' ' ']
 [' ' ' ' ' ']] X in (1, 1), id=0_3 is best ←
 	(assumption) By definition a node is "best" if it's "better than" than all "siblings"
	∧ 
	[['O' ' ' ' ']
	 [' ' 'X' ' ']
	 [' ' ' ' ' ']] X in (1, 1), id=0_3 has siblings = 
	[['O' 'X' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] X in (0, 1), id=0_0, 
	[['O' ' ' 'X']
	 [' ' ' ' ' ']
	 [' ' ' ' ' ']] X in (0, 2), id=0_1, 
	[['O' ' ' ' ']
	 ['X' ' ' ' ']
	 [' ' ' ' ' ']] X in (1, 0), id=0_2, 
	[['O' ' ' ' ']
	 [' ' ' ' 'X']
	 [' ' ' ' ' ']] X in (1, 2), id=0_4, 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 ['X' ' ' ' ']] X in (2, 0), id=0_5, 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' 'X' ' ']] X in (2, 1), id=0_6, 
	[['O' ' ' ' ']
	 [' ' ' ' ' ']
	 [' ' ' ' 'X']] X in (2, 2), id=0_7 ←
	 		(assumption) Definition of "siblings" is node.parent.children excluding node
	∧ 
	[['O' ' ' ' ']
	 [' ' 'X' ' ']
	 [' ' ' ' ' ']] X in (1, 1), id=0_3 is better than them ←
	 		(assumption) By